# Lab 9: Production LLM Deployment with Tool Calling (vLLM + FastAPI)

**Goal:**  
Build a production-grade LLM service with tool calling capabilities using vLLM for inference and FastAPI for orchestration.

## Architecture Overview

```
Client Request
     ↓
FastAPI Orchestration Layer (handles tool calling logic)
     ↓
vLLM Server (high-performance inference)
     ↓
Tool Execution (weather API, calculator, database, etc.)
     ↓
Response with tool results
```

## Why This Two-Tier Architecture?

- **vLLM**: Production-grade inference with v1 engine (1.5-2x faster than v0)
  - Continuous batching
  - PagedAttention
  - Auto-optimized scheduling (don't use v0 flags!)
- **FastAPI**: Lightweight orchestration for tool calling business logic
  - Tool execution
  - Multi-tenant routing
  - Business logic
- **Separation of concerns**: Inference engine vs application logic
- **Independent scaling**: Scale vLLM on GPU, FastAPI on cheap CPU
- **Cost optimization**: GPU only used for inference (~100ms), not waiting for tool API calls (seconds)

**⚠️ vLLM v1 Engine (2025)**: Default since v0.8.0. Don't add `--enable-chunked-prefill` or `--num-scheduler-steps` - they force v0 fallback and reduce performance.

This is how companies like Anthropic, OpenAI, and major LLM platforms structure their tool-calling systems.

**Time:** ~40 minutes

In [ ]:
# Install Unsloth using the official auto-install script
# This automatically detects your environment and installs the correct version
!wget -qO- https://raw.githubusercontent.com/unslothai/unsloth/main/unsloth/_auto_install.py | python -

# Alternative manual installation if auto-install fails:
# !pip install --upgrade pip
# !pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
# !pip install "unsloth_zoo @ git+https://github.com/unslothai/unsloth-zoo.git"

print("✅ Unsloth installation complete! Now restart runtime before proceeding.")
print("⚠️ IMPORTANT: Use GPU runtime, not TPU! Unsloth requires CUDA GPU.")

> **⚠️ CRITICAL IMPORT ORDER**: 
> - Always import `unsloth` FIRST before any other ML libraries
> - This prevents weights/biases initialization errors
> - Example: `from unsloth import FastLanguageModel` then `import torch`


### Step 1: Build an OpenAI-compatible FastAPI server with Tool Calling support

**Documentation:**
- OpenAI Tool Calling: https://platform.openai.com/docs/guides/tools
- FastAPI advanced features: https://fastapi.tiangolo.com/advanced/
- JSON Schema: https://json-schema.org/learn/


In [ ]:
# TODO: Import (CRITICAL: Import unsloth FIRST!) necessary libraries (FastAPI, Pydantic, torch, unsloth, json)# TODO: Load your optimized model using FastLanguageModel.from_pretrained()# TODO: Define Pydantic models for the API:#   - ChatMessage (role: str, content: str)#   - ToolSchema (name: str, description: str, parameters: Dict[str, Any])#   - ChatRequest (model: str, messages: List[ChatMessage], temperature: float, max_tokens: int, tools: Optional[List[ToolSchema]])#   - ChatResponse (id: str, object: str, choices: List[Dict], model: str)# TODO: Define a sample tool function (e.g., get_current_weather)# Example:# def get_current_weather(location: str, unit: str = "celsius") -> str:#     return f"The weather in {location} is 72° {unit.upper()}."# TODO: Create FastAPI app instance# TODO: Create POST endpoint at "/v1/chat/completions" that:#   1. Receives ChatRequest#   2. If req.tools is not None:#      a. Parse tools definitions#      b. Generate response from model (may include tool call)#      c. If response contains tool call:#         - Parse tool name and arguments#         - Execute the Python function#         - Append tool result as a message#         - Rerun chat with tool result included#      d. Return final assistant answer#   3. Otherwise, handle as normal chat# TODO: Add instructions for running the server# Hint: Use uvicorn to run the app in the background

### Step 2: Test the API with an OpenAI client

**Documentation:**
- OpenAI function calling: https://platform.openai.com/docs/guides/function-calling
- OpenAI Python client: https://github.com/openai/openai-python


In [ ]:
# This cell shows how to test your FastAPI service once it's running locally.
# Please start the FastAPI server in a separate process or notebook cell, then run the following code:

# TODO: Import OpenAI client

# TODO: Set BASE_URL to your local server

# TODO: Create OpenAI client with custom base_url

# TODO: Define tools/functions array with tool schema:
#   - name: "get_current_weather"
#   - description: "Get the current weather for a given location and unit"
#   - parameters: JSON schema with properties for location and unit

# TODO: Make a chat completion request with:
#   - model name
#   - messages with user query (e.g., "What's the weather in San Francisco?")
#   - tools parameter
#   - tool_choice="auto"
#   - temperature=0.0

# TODO: Print the response content

# Note: Observe whether the model chose to call the tool and how the final answer incorporates the tool's result


## Production Deployment: Multi-Service Architecture

To deploy the two-tier architecture on AWS ECS, use docker-compose locally and separate ECS services in production:

**Documentation:**
- vLLM documentation: https://docs.vllm.ai/
- FastAPI deployment: https://fastapi.tiangolo.com/deployment/
- Docker Compose: https://docs.docker.com/compose/

### docker-compose.yml

```yaml
version: '3.8'

services:
  # vLLM inference engine (GPU, v1 engine)
  vllm:
    image: vllm/vllm-openai:latest
    command:
      - python
      - -m
      - vllm.entrypoints.openai.api_server
      - --model
      - /app/model
      - --host
      - 0.0.0.0
      - --port
      - '8000'
      - --dtype
      - float16
      - --gpu-memory-utilization
      - '0.9'
      # ⚠️ DON'T add --enable-chunked-prefill or --num-scheduler-steps
      # These force v0 fallback. v1 engine auto-optimizes.
    volumes:
      - ./vllm_model:/app/model
    deploy:
      resources:
        reservations:
          devices:
            - driver: nvidia
              count: 1
              capabilities: [gpu]
    networks:
      - llm-network

  # FastAPI orchestration layer (CPU)
  orchestrator:
    build:
      context: .
      dockerfile: Dockerfile.orchestrator
    ports:
      - '8001:8001'
    environment:
      - VLLM_URL=http://vllm:8000
    depends_on:
      - vllm
    networks:
      - llm-network

networks:
  llm-network:
    driver: bridge
```

### Dockerfile.orchestrator

```dockerfile
FROM python:3.11-slim

WORKDIR /app

RUN pip install fastapi uvicorn httpx pydantic

COPY tool_orchestrator.py .

EXPOSE 8001

CMD ["uvicorn", "tool_orchestrator:app", "--host", "0.0.0.0", "--port", "8001"]
```

### AWS ECS Deployment Strategy

Deploy as **two separate services**:

1. **vLLM Service** (GPU instances - g5.xlarge+):
   - Runs on GPU-enabled EC2 instances
   - Uses vLLM v1 engine for auto-optimization
   - Registered with AWS Cloud Map for service discovery
   - Scale vertically (bigger GPUs) for larger models

2. **Orchestrator Service** (CPU instances - Fargate or c5.xlarge):
   - Runs on cheap CPU (Fargate or EC2)
   - Scale horizontally (2+ instances) for high request volume
   - Handles all business logic and tool execution
   - Can deploy updates without touching vLLM

**Benefits:**
- **Cost**: GPU only for inference, CPU for orchestration (70% cost savings)
- **Scalability**: Scale each tier independently based on load
- **Flexibility**: Update tools without redeploying GPU instances
- **Performance**: vLLM v1 engine provides 1.5-2x throughput automatically

## Reflection

### Performance Analysis
- How many tokens did the tool call use compared to a normal chat response?
- Did the model choose to call the tool or answer directly? Why or why not?
- What is your measured latency end-to-end (user → model → tool → model → user)?
- How could you extend this pattern to allow multiple tools or tool chaining?

### Production Considerations
- **Latency breakdown**: 
  - vLLM inference: ~100-500ms per call
  - Tool execution: varies (API calls 100ms-2s, DB queries 10-100ms)
  - Total for tool calling: 2-3x normal chat latency (acceptable trade-off)

- **vLLM v1 vs v0 engine**: 
  - v1 provides 1.5-2x better throughput automatically
  - Don't use `--enable-chunked-prefill` or `--num-scheduler-steps` (forces v0 fallback)
  - v1 auto-tunes these parameters for your workload

- **Scaling strategy**:
  - **vLLM** (GPU): Scale vertically (g5.xlarge → g5.12xlarge) for larger models
  - **Orchestrator** (CPU): Scale horizontally (2+ Fargate instances) for high traffic
  - Monitor `vllm:num_requests_waiting` metric for autoscaling trigger

- **Cost optimization**:
  - GPU only used during inference (~100ms), not during tool execution (seconds)
  - 70% cost savings vs running everything on GPU
  - Use Spot instances for additional 55-60% savings

- **Enterprise patterns**:
  - Ray Serve for >1k QPS with multi-model routing
  - LoRA multi-tenancy for per-customer models
  - Multi-region deployment for global low-latency